In [ ]:
import warnings
import numpy as np
import os
import pandas as pd
import openpyxl
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset  
from torch.utils.data.dataset import random_split
import torchvision.transforms as transforms
import transformers
from transformers import BertForSequenceClassification , Trainer , TrainingArguments
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from datasets import load_metric
from transformers import Trainer
from datasets import load_metric
import pandas as pd
import torch

In [ ]:

df = pd.read_csv("embed_data")
df = df.loc[(df.label == "standard") | (df.label == "requirement")]
print(df.groupby("label").count())
print(df.head())

text = df.text.tolist()
labels = df.label.map({'standard': 1, 'requirement': 0}).tolist()

In [ ]:
requirements = df.text.tolist()
labels = df.label.map({'standard': 1, 'requirement': 0}).tolist()
#print(labels)

In [ ]:
train_requs , test_requs , train_labels , test_labels = train_test_split (
requirements , labels , random_state =500 , test_size =.2)

train_requs , val_requs , train_labels , val_labels = train_test_split (
train_requs , train_labels , random_state =501 , test_size =.1)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer ( train_requs , truncation = True , padding = True )
val_encodings = tokenizer ( val_requs , truncation = True , padding = True )
test_encodings = tokenizer ( test_requs , truncation = True , padding = True )



In [ ]:
class RequDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RequDataset(train_encodings, train_labels)
val_dataset = RequDataset(val_encodings, val_labels)
test_dataset = RequDataset(test_encodings, test_labels)


In [ ]:
import numpy as np
import torch
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback, get_linear_schedule_with_warmup
from datasets import load_metric
from datasets import load_dataset
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create collapsible output widget
output = widgets.Output()
output_collapsible = widgets.Accordion([output])
output_collapsible.set_title(0, 'Model Output')
display(output_collapsible)

# Function to display content in the output widget
def display_in_output(content):
    with output:
        clear_output(wait=True)
        display(content)



# Define your training arguments first
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_strategy='epoch',
    load_best_model_at_end=True,
)

# Define your model and optimizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)

# Load the metric
metric = load_metric("matthews_correlation")

# Define your compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels_string = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels_string)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Define your early stopping callback
early_stopping = EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.01)

# Manually calculate the total number of training steps
total_train_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

# Define your learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_train_steps * training_args.warmup_ratio),
    num_training_steps=total_train_steps
)

# Initialize lists to store training and evaluation metrics
train_losses = []
eval_losses = []
eval_matthews_corrs = []

# Add the early stopping callback to the trainer
trainer.add_callback(early_stopping)

# Custom training loop
for epoch in range(training_args.num_train_epochs):
    # Training
    trainer.train()

    # Validation
    trainer.evaluate()

    # Create a collapsible widget for model output
    model_output = widgets.Output()
    model_output_collapsible = widgets.Accordion([model_output])
    model_output_collapsible.set_title(0, f'Epoch {epoch + 1}/{training_args.num_train_epochs}')
    
    # Function to display content in the model_output widget
    def display_in_model_output(content):
        with model_output:
            clear_output(wait=True)
            display(content)
    
    # Get training and evaluation metrics
    train_loss = trainer.callback_metrics['train_loss']
    eval_loss = trainer.callback_metrics['eval_loss']
    eval_matthews_corr = trainer.callback_metrics['eval_matthews_correlation']
    
    # Append metrics to lists
    train_losses.append(train_loss)
    eval_losses.append(eval_loss)
    eval_matthews_corrs.append(eval_matthews_corr)
    
    # Display training and validation metrics in model output
    display_in_model_output(f"Training Loss: {train_loss:.4f}")
    display_in_model_output(f"Validation Loss: {eval_loss:.4f}")
    display_in_model_output(f"Matthews Correlation: {eval_matthews_corr:.4f}")
    
    # Display the collapsible model output widget
    display(model_output_collapsible)

    # Check for early stopping
    if early_stopping.early_stopping.should_stop:
        display_in_output("Early stopping triggered.")
        break

    # Update the learning rate using the scheduler
    for _ in range(total_train_steps):
        scheduler.step()

# Access training and evaluation metrics from the lists
final_train_loss = train_losses[-1]
final_eval_loss = eval_losses[-1]
final_eval_matthews_corr = eval_matthews_corrs[-1]


# Print final metrics
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_eval_loss:.4f}")
print(f"Final Matthews Correlation: {final_eval_matthews_corr:.4f}")



In [ ]:

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
from sklearn . metrics import matthews_corrcoef as mcc
mcc_test_labels = [ label for label in test_labels ]
mcc_preds = [ label for label in preds . tolist () ]
print ( mcc_preds )
print ( mcc_test_labels )
print ( mcc ( mcc_test_labels , mcc_preds ) )

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(test_labels, preds))
print(metrics.classification_report(test_labels, preds, target_names=['Standard', 'Requirement']))

In [ ]:
# Save the model's weights in the "savedmodels" folder
model.save_pretrained('savedmodels')

# Optionally, save the model's configuration and tokenizer in the same folder
model.config.save_pretrained('savedmodels')
tokenizer.save_pretrained('savedmodels')
